<h3 style="text-align: center;">EAS-503-Group7-ML-Project</h3>
<h1 style="text-align: center">Video Games Sales Prediction</h1>
<h3 style="text-align: center">Presentors: Rohan Shrikant Thorat (50557738), Nikhil Nitin Gokhale (50560271), Prathamesh Ravindra Varhadpande (50559586)</h2>
<h3 style="text-align: center">Mentor: Prof. MOHAMMAD ZIA (PhD)</h3>

<h4>About Dataset:</h4>
<p>This dataset contains a list of video games with sales greater than 100,000 copies. It was generated by a scrape of vgchartz.com.

Fields include

Rank - Ranking of overall sales

Name - The games name

Platform - Platform of the games release (i.e. PC,PS4, etc.)

Year - Year of the game's release

Genre - Genre of the game

Publisher - Publisher of the game

NA_Sales - Sales in North America (in millions)

EU_Sales - Sales in Europe (in millions)

JP_Sales - Sales in Japan (in millions)

Other_Sales - Sales in the rest of the world (in millions)

Global_Sales - Total worldwide sales.

Critic_score - Aggregate score compiled by Metacritic staff

Critic_count - The number of critics used in coming up with the Critic_score

User_score - Score by Metacritic's subscribers

User_count - Number of users who gave the user_score

Developer - Party responsible for creating the game

Rating - The ESRB ratings

There are 16,598 records. 

Link to dataset : https://www.kaggle.com/datasets/sidtwr/videogames-sales-dataset
</p>

<h4>Step 1: (A) Creating Normalized Database</h4>
<p>a. Created SQLite Database named "videogamessales"</p> 
<p>b. Created three normalized tables : Games, Sales and User_Reviews</p>
<p>c. Games table include - Name (Primary Key), Platform, Year_of_Release, Genre, Publisher and Rating</p>
<p>d. Sales table include - Name (Foreign Key), NA_Sales, EU_Sales, JP_Sales, Other_Sales and Global_Sales</p>
<p>In Sales table we used composite key as primary key</p>
<p>Composite key (Name, NA_Sales, EU_Sales, JP_Sales, Other_Sales and Global_Sales)</p>
<p>e. User_Reviews table includes - Name (Primary Key) (Foreign Key from Games table), User_Score, User_Count</p>

In [ ]:
import csv
import sqlite3

# Connecting to SQLite database named "videogamessales.db"
conn = sqlite3.connect('videogamessales.db')
cursor = conn.cursor()

# Creating tables
# Creating Games table
conn.execute('''CREATE TABLE IF NOT EXISTS Games (
                    Name TEXT PRIMARY KEY,
                    Platform TEXT,
                    Year_of_Release REAL,
                    Genre TEXT,
                    Publisher TEXT,
                    Rating TEXT
                    )''')

# Creating Sales table
conn.execute('''CREATE TABLE IF NOT EXISTS Sales (
                    Name TEXT,
                    NA_Sales REAL,
                    EU_Sales REAL,
                    JP_Sales REAL,
                    Other_Sales REAL,
                    Global_Sales REAL,
                    PRIMARY KEY (Name, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales),
                    FOREIGN KEY (Name) REFERENCES Games(Name)
                    )''')

# Creating User_Reviews table
conn.execute('''CREATE TABLE IF NOT EXISTS User_Reviews (
                    Name TEXT PRIMARY KEY,
                    User_Score REAL,
                    User_Count REAL,
                    FOREIGN KEY (Name) REFERENCES Games(Name)
                    )''')

with open('datasets/VideoGamesSalesCleaned.csv', 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    games_data = []
    sales_data = []
    user_reviews_data = []
    for row in reader:
        games_data.append((row['Name'], row['Platform'], row['Year_of_Release'], row['Genre'], row['Publisher'], row['Rating']))
        sales_data.append((row['Name'], row['NA_Sales'], row['EU_Sales'], row['JP_Sales'], row['Other_Sales'], row['Global_Sales']))
        user_reviews_data.append((row['Name'], row['User_Score'], row['User_Count']))

# Inserting Games data
cursor.executemany('''INSERT INTO Games (Name, Platform, Year_of_Release, Genre, Publisher, Rating) 
                    VALUES (?, ?, ?, ?, ?, ?)''', games_data)

# Inserting Sales data
cursor.executemany('''INSERT INTO Sales (Name, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales) VALUES (?, ?, ?, ?, ?, ?)''', sales_data)

# Inserting User Reviews data
cursor.executemany('''INSERT INTO User_Reviews (Name, User_Score, User_Count) VALUES (?, ?, ?)''',
                    user_reviews_data)

# Committing changes and closing connection
conn.commit()
conn.close()

print("Database and tables created successfully.")

<h4>Step 1: (B) Joining the Normalized Database into a Pandas Dataframe</h4>

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('videogamessales.db')

# SQL query to fetch data using JOINs
sql_query = '''
    SELECT g.Name, g.Platform, g.Year_of_Release, g.Genre, g.Publisher, g.Rating,
       s.NA_Sales, s.EU_Sales, s.JP_Sales, s.Other_Sales, s.Global_Sales,
       u.User_Score, u.User_Count
FROM Games g
LEFT JOIN Sales s ON g.Name = s.Name
LEFT JOIN User_Reviews u ON g.Name = u.Name
'''
# Execute the SQL query and fetch data into a Pandas DataFrame
df = pd.read_sql_query(sql_query, conn)

# Close the connection
conn.close()

# Display the DataFrame
print(df)